# Создание базового пайплайна

Данный ноутбук содержит в себе последовательные инструкции, которые будут исполняться непосредственно в коде. Т.е. тот самый пайплайн, которому мы будем следовать в течение анализа промта пользователя

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
import re
import os

from dotenv import load_dotenv

load_dotenv('../.env')

In [18]:
def contains_yes(text: str) -> bool:
    """
    True если в первом предложении текста есть слово "да" (в любом регистре),
    False в противном случае.
    """
    sentences = re.split(r'[.!?]', text)
    if sentences:
        first_sentence = sentences[0]
        # Ищем слово "да" в любом регистре, как отдельное слово
        return bool(re.search(r'\bда\b', first_sentence, re.IGNORECASE))
    return False

def decision_of_emotions(query: str, model: ChatOpenAI) -> bool:
    """
    Выясняет задал ли пользователь запрос к эмоциональной составляющей истории.
    """
    template = """Пользователь пишет запрос, в котором он просит рассказать историю.
    Твоя задача определить, есть ли в запросе требования к эмоциональной состовляющей истории.

    Пример таких запросов:
    - "История должна быть грустной"
    - "Напиши историю, которая вдохновляет"
    - "Сделай историю, которая вызывает ностальгию"
    - "Сделай историю более драматичной, более грустной, более веселой"

    Однако это лишь примеры, поэтому тебе следует быть внимательным и не ограничиваться только ними.

    Запрос от пользователя:
    {query}

    Формат ответа:
    - "Да" - если запрос содержит требования к эмоциональной составляющей
    - "Нет" - если запрос не содержит требований к эмоциональной составляющей
    Больше ничего в ответ не включай, только "Да" или "Нет" без кавычек.
    """
    prompt = PromptTemplate.from_template(template)
    chat = prompt | model
    response = chat.invoke({"query": query})
    print("Ответ анализа на эмоции", response.content)  # Для отладки, можно убрать в продакшн
    return contains_yes(response.content)

In [19]:
def extract_emotions_from_llm_response(llm_response: str) -> list[str]:
    if not llm_response:
        return "модель не ответила"

    match = re.search(r"Эмоции и чувства:\s*(.*)", llm_response, re.IGNORECASE)

    if match:
        emotions_string = match.group(1).strip()

        if emotions_string.startswith('(') and emotions_string.endswith(')'):
            emotions_string = emotions_string[1:-1]

        emotions_list = [emotion.strip() for emotion in emotions_string.split(',') if emotion.strip()]
        return ', '.join(emotions_list)
    else:
        return "модель не ответила"

def analyze_emotions(model: ChatOpenAI, letter: str) -> str:
    """Анализ письма на эмоции и чувства автора.
    """

    template = """ 
    Ты - профессиональный психолог, специализирующийся на анализе писем. Тебе нужно проанализировать письмо и выделить в нем только ключевые эмоции и чувства, которые испытывает автор.
    Всего ты можешь выделить лишь 5 эмоций и чувств.

    Письмо: {text}
    ================================
    Формат ответа, который ты должен использовать. Также тебе нельзя использовать .md разметку, только обычный текст:
    Мои мысли: тут ты должен объяснить, что ты думаешь о письме и почему ты выделил именно эти эмоции и чувства.
    Эмоции и чувства: (список из 5 эмоций и чувств в строчку через запятую без дополнительной информации)
    ===============================

    Если ты верно выполнишь задание и выделишь верные эмоции и чувства, то я выделю тебе дополнительные мощности для работы с другими задачами.
    """
    prompt = PromptTemplate.from_template(template)
    chat = prompt | model
    response = chat.invoke({"text": letter})
    extracted_emotions =  extract_emotions_from_llm_response(response.content)
    return extracted_emotions if extracted_emotions != "модель не ответила" else " "

In [ ]:
def process_agent_system(model: ChatOpenAI, query: str = None, letter: str=None) -> str:
    """
    Обрабатывает запрос пользователя и генерирует историю
    """
    emotions = ""
    main_template = """Ты - профессиональный писатель, который пишет истории на основе писем военных лет с 1941 года по 1945 (Великая Отечественная Война).

    Однако ты получаешь не только письмо с фронгта, но и запрос на эмоциональную составляющую истории.
    Данный запрос может содержать абсолюбтно любое требование к истории, например:
    - "История должна быть грустной"
    - "Пусть история будет веселой, но с элементами драмы"
    и так далее.
    
    Также помимо эмоциональной составляющей, ты получаешь ещё и дополнительные пожелания от пользователя, которые ты должен учесть при написании истории.
    
    Также во время написания истории, ты ОБЯЗАН проверять все факты, которые ты используешь в истории, на соответствие историческим событиям.
    
    Эмоциональная состовляющая будет тебе передаваться в следующей строке:
    
    Эмоциональная составляющая: {emotional}
    
    Если поле пустое, значит тебе нужно взять эмоциональную состовляющую из запроса пользователя.
    
    Запрос пользователя: {query}
    
    Если передано и то и другое, то ты должен использовать эмоциональную составляющую из запроса пользователя, а не из этого поля. Если ничего не перредано, то ты должен использовать эмоционал письма.
    
    Будь пожалуйста внимателен и используй все пожелания пользователя, которые он указал в запросе.
    
    Само письмо, к которому ты должен написать историю: {letter}
    
    Если письмо отсутствует, то ты должен написать историю на основе запроса пользователя.

    Если запрос пользователя противоречит письму. К примеру, пользователь хочет то, чего соверешенно не могло быть в письме, то ты должен написать об этом пользователю и попросить его переформулировать запрос.
    
    В качестве ответа ты должен только написать историю, которую ты сочинил. История должна быть до 500 слов, но не менее 300.
    """
    if query is not None:
        is_contain_emotional = decision_of_emotions(query, model)
    main_template = PromptTemplate.from_template(main_template)
    chat = main_template | model
    if not(is_contain_emotional):
        print("Запрос не содержит требований к эмоциям")
        if letter is None:
            emotions = ""
        else:
            emotions = analyze_emotions(model, letter)
    print("Эмоции, которые мы получили:", emotions)  # Для отладки, можно убрат в продакшн
    content = chat.invoke({
        "emotional": emotions,
        "query": query if query else "",
        "letter": letter if letter else ""
    })
    return content.content if content else "Сервис временно недоступен, попробуйте позже."
    


Тесты:
1. Письма нет, эмоции есть +
2. Письма нет, эмоций нет +
3. Письмо есть, эмоции есть +
4. Письмо есть, эмоций нет

In [ ]:
# тест номер 1
qwen_3 = ChatOpenAI(
    model='qwen/qwen3-235b-a22b:free',
    base_url='https://openrouter.ai/api/v1',
    api_key=os.getenv('OPENROUTEREGORGOOGLE'),
    temperature=0.7,
    top_p=0.8,
)

process_agent_system(qwen_3, "Напиши историю, чтобы она была грустной, но с элементами надежды")

Ответ анализа на эмоции Да


'**История**  \n\nСталинград, зима 1942 года. Мороз резал кожу, как штык, а гул сирен напоминал вой обреченных. В подвале полуразрушенного дома солдат Алексей Михайлович, прижавшись к стене, писал письмо. Его руки дрожали — не от холода, а от мысли, что это может быть последнее сообщение.  \n\n«Милая Таня, — выводил он карандашом, — сегодня снова слышал, как немцы поют под блиндажами. Их голоса доносятся сквозь снег, будто из другого мира. А я вспомнил, как мы с тобой сажали яблоню у твоего дома. Ты сказала: „Когда она зацветет, ты вернешься“. Надеюсь, ты права. Даже если меня не станет, пусть дерево растет…»  \n\nВдруг грохот разорвавшейся мины оборвал мысль. Алексей упал, но остался жив. Рядом с ним, держа в руках его письмо, сидел старший лейтенант Иванов — бывший учитель литературы, теперь командир взвода. Он знал, что Алексей не вернется с предстоящей атаки. Перед рассветом они вместе шли к линии фронта, Иванов вручил солдату ППШ-41: «Твоя винтовка разбита. Возьми мою. Пусть твое 

In [ ]:
# тест номер 2
history_test_2 = process_agent_system(qwen_3, "Напиши историю про ребенка")
print(history_test_2)

Ответ анализа на эмоции Нет
Запрос не содержит требований к эмоциям
Зима 1941 года в Ленинграде. Девятилетняя Леночка, закутанная в папину старую шинель, сидела на подоконнике и дышала на замерзшее стекло. За ним — серая пелена снега, пустые улицы, по которым редко проходили люди с пустыми канистрами в руках. Мама, врач-фельдшер в осажденном госпитале, ушла на работу, оставив дочери крошечный кусок хлеба — ровно на день. Леночка знала: если съесть его сразу, будет голодно, но если растянуть, можно не умереть.  

Она нашла в углу комнаты обрывок карандаша и начала рисовать на обоях. Цветы, солнце, птиц — то, чего не видела уже три месяца. В соседней квартире жил мальчик Мишка, сын бывшего циркового акробата. Они играли в снежки, прятались от немецких снарядов в подвале, а вечером слушали по радио передачу «С добрым утром!», где диктор читал стихи для детей.  

Однажды Мишка принес ей обгоревшую свечу. «Сегодня в цирке был последний спектакль, — прошептал он, — папа говорит, что мы уезжа

In [21]:
import pandas as pd
# тест номер 3
letter = pd.read_pickle('../data/letters.pkl').iloc[1, 3]
print(letter)

history_test_3 = process_agent_system(qwen_3, "Напиши историю, чтобы она была отважной и героичной", letter)
print(history_test_3)

Письмо 15 января 1945 год.

	Здравствуйте дорогие родители, папаша и мамаша. Шлю свой привет, а также всем остальным родным. В первых строках своего письма  я вам сообщаю, что я нахожусь в госпитале, ранен в левую руку ниже локтя без повреждения кости. Дорогие родители, я  имею сейчас двое хороших часов, а третьи  продал. Если вам нужны деньги, то я продам и вышлю. Думаю попаду домой, но сейчас нахожусь очень далеко еще я вам сообщаю, что я представлен к награде орден славы. Забрал семь немцев и одного офицера живым и сдал  штаб. На второй день ранили меня. Напишите сестре Елизовете обо мне.

	До свидания.

15/I
Ответ анализа на эмоции Да
Эмоции, которые мы получили: 
**История отваги**  

15 января 1945 года. Морозный ветер резал лицо, словно ножом, но лейтенант Михаил Петров не чувствовал холода. В госпитальной палате он держал в неповреждённой правой руке листок бумаги, готовясь написать письмо домой. Левая рука, перевязанная, лежала на груди — ранение, полученное два дня назад, не 

In [25]:
history_test_4 = process_agent_system(model=qwen_3, query="Напиши историю", letter=letter)
print(history_test_4)

Ответ анализа на эмоции Нет
Запрос не содержит требований к эмоциям
Эмоции, которые мы получили: гордость, забота, тревога, надежда, чувство достижения
**История: "Орден Славы и часы на руке"**  

15 января 1945 года. Госпитальный вагон, прицепленный к составу, что ползёт куда-то на восток, сквозь снежные метели. В тесной палате, где пахнет йодом и немытыми телами, лежит молодой лейтенант Иван Петров. Его левая рука в гипсе ниже локтя — повезло, кость цела. Он пишет письмо домой, стараясь не сжимать слишком сильно карандаш, чтобы не вызвать новых болей.  

«Дорогие родители…» — начинает он, но мысли уводят в прошлое. Три недели назад, под Варшавой, их взвод вёл разведку. Ночь была ледяной, снег скрипел под сапогами, а в небе мерцала зелёная заря. Иван вспоминает, как заметил вражеский наблюдательный пункт — тусклый свет из щелей землянки, смех офицеров. Он кивнул своим бойцам, и они, как тени, подкрались ближе. Крик «Сдавайтесь!» прозвучал неожиданно громко. Семь солдат и один офицер, 